In [ ]:
import pandas as pd
import numpy as np
from Bio import SeqIO
strains = []
genomes = []
for record in SeqIO.parse('C:/Users/srava/BedfordProjects/zika/results/aligned.fasta', "fasta"):
        strains.append(str(record.id))
        genomes.append(str(record.seq))

In [ ]:
dataa = pd.read_csv("DistanceMatrix")

In [ ]:
pairwise_distance = dataa.to_numpy()[np.triu_indices(486, k = 1)]

In [ ]:
pairwise_distance

In [ ]:
dataa_subset = dataa.loc[0:49, 'strain 1':'strain 50']

In [ ]:
pairwise_distance_subset = dataa.to_numpy()[np.tril_indices(len(genomes), k = 1)]

In [ ]:
from sklearn.manifold import TSNE
embedding = TSNE(n_components=2,metric='precomputed',perplexity = 25.95)
X_transformed = embedding.fit_transform(dataa)

In [ ]:
MDS_df = pd.DataFrame(X_transformed,columns=['cluster ' + str(i) for i in range(1,3)])
print(MDS_df.head())

In [ ]:
#merging my final dataframe with their regions and strain names
import pandas as pd
metadata_df = pd.read_csv(r'C:\Users\srava\BedfordProjects\zika\results\metadata.tsv', delimiter='\t')

In [ ]:
metadata_df.head()

In [ ]:
strains_df = pd.DataFrame(strains)
strains_df.columns = ['strain']

In [ ]:
result_strains = pd.merge(strains_df,metadata_df, on='strain')

In [ ]:
result_strains.head()

In [ ]:
finalDf = pd.concat([MDS_df, result_strains[['region','country','strain']]], axis = 1)

In [ ]:
finalDf

In [ ]:
#There are 519 strains up until here

In [ ]:
import altair as alt
from augur.utils import json_to_tree
import json
import pandas as pd

In [ ]:
# Import modules
with open('Data/zika_tree.json') as fh:
    json_tree_handle = json.load(fh)
#json_tree_handle = json.loads(requests.get("http://data.nextstrain.org/zika_tree.json").text)
# Download tree JSON file used by auspice. Despite its filename extension, the data are compressed.
#http = urllib3.PoolManager()
# = http.request('GET', "")


In [ ]:
tree = json_to_tree(json_tree_handle)

In [ ]:
tree

In [ ]:
node_data = [
    {
        "strain": node.name,
        "date": node.attr["num_date"],
        "y": node.yvalue,
        "region": node.attr["region"],
        "country": node.attr["country"],
        "parent_date": node.parent is not None and node.parent.attr["num_date"] or node.attr["num_date"],
        "parent_y": node.parent is not None and node.parent.yvalue or node.yvalue
    }
    for node in tree.find_clades(terminal=True)
]

In [ ]:
node_data[10]

In [ ]:
node_df = pd.DataFrame(node_data)

In [ ]:
node_df.head()
#there are 517 strains here

In [ ]:
node_df["y"] = node_df["y"].max() - node_df["y"]

In [ ]:
node_df["parent_y"] = node_df["parent_y"].max() - node_df["parent_y"]

In [ ]:
node_df.shape

In [ ]:
node_df.head()

In [ ]:
node_df["region"].unique()

In [ ]:
finalDf["region"].unique()

In [ ]:
merged_df = pd.merge(finalDf,node_df,on=["strain","region"]).dropna()

In [ ]:
merged_df
#this has 485 strains

# Pairwise vs. Euclidean distance scatterplot

In [ ]:
#change pandas to numpy for euclidean and pairwise
#plot the points and draw the least squares regression line
#print out r
#do it for T-SNE, UMAP, MDS, all of them and add it as a panel to all the graphs
#start putting together a more cohesive github pages to look at all the panels

#make a final project notebook and start putting it together in a cohesive way
#try for flu clades (Color by clade instead of region) and see how it works

In [ ]:
#work on making this work

import pandas as pd
import numpy as np
from Bio import SeqIO
strains = []
genomes = []
for record in SeqIO.parse('C:/Users/srava/BedfordProjects/zika/results/aligned.fasta', "fasta"):
        strains.append(str(record.id))
        genomes.append(str(record.seq))

In [ ]:
len(strains)
print(len(genomes))

In [ ]:
strains[:5]

In [ ]:
merged_df.head()

In [ ]:
#recalculate the pairwise distance here with merged df (add column of genomes and run pairwise distance again)

In [ ]:
genomes_df = pd.DataFrame(genomes)
genomes_df.columns = ['genomes']

In [ ]:
strains_df = pd.DataFrame(strains)
strains_df.columns = ['strain']

In [ ]:
df_merged = genomes_df.merge(strains_df, how='outer', left_index=True, right_index=True)

In [ ]:
merged2_df = df_merged.merge(merged_df, how = 'inner', on = 'strain')
merged2_df = merged2_df[['genomes','strain']]
merged2_df.head()

In [ ]:
genomes_numpy = merged2_df[['genomes']].to_numpy()
genomes = genomes_numpy.tolist()
genomes = list(genomes_numpy)

In [ ]:
# Return the Hamming distance between string1 and string2.
# string1 and string2 should be the same length.
def hamming_distance(array1, array2): 
    # Start with a distance of zero, and count up
    distance = 0
    # Loop over the indices of the string
    L = len(array1)
    for i in range(L):
        # Add 1 to the distance if these two characters are not equal
        if array1[i] != array2[i] and array1[i] != 'N' and array2[i] != 'N':
            distance += 1
    # Return the final count of differences
    return distance

In [ ]:
#Using the Hamming distance to create a similarity matrix
numbers = []
matrixOfNum = []
for i in merged2_df.index:
    for j in merged2_df.index:
        num = hamming_distance(merged2_df['genomes'][i],merged2_df['genomes'][j])
        numbers.append(num)
    matrixOfNum.append(numbers)
    numbers = []
dataa = pd.DataFrame(matrixOfNum, columns = ["strain " + str(i) for i in range(1,len(matrixOfNum) + 1)], dtype="float")
print(dataa)

In [ ]:
#Read in correct matrix
merged_df

In [ ]:
dataa = dataa.to_csv("DistanceMatrixPairwiseEuclidean",sep=',',index=False)

In [ ]:
dataa.shape

In [ ]:
import numpy as np
pairwise_distance_array = np.array(dataa)[np.triu_indices(486, k = 1)]

In [ ]:
len(pairwise_distance_array)

In [ ]:
pairwise_df = pd.DataFrame(pairwise_distance_array)

In [ ]:
from scipy.spatial.distance import squareform, pdist
euclidean_distance_array = pdist(merged_df[['cluster 1','cluster 2']])

In [ ]:
euclidean_distance_array

In [ ]:
len(euclidean_distance_array)

In [ ]:
euclidean_df = pd.DataFrame(euclidean_distance_array)

In [ ]:
final_df = pairwise_df.merge(euclidean_df,how='inner',left_index=True, right_index=True)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
alt.Chart(final_df).mark_circle(size=60).encode(
    x=alt.X('0_x',title = "pairwise distance"),
    y=alt.X('0_y', title = "Euclidean distance"),
    tooltip=['0_x', '0_y']
).properties(title="pairwise vs. Euclidean scatterplot: T-SNE")

In [ ]:
dataa = pd.read_csv("DistanceMatrixPairwiseEuclidean")

In [ ]:
from sklearn.manifold import MDS
embedding = MDS(n_components=10,metric=True,dissimilarity='precomputed')
X_transformed = embedding.fit_transform(dataa)

In [ ]:
MDS_df = pd.DataFrame(X_transformed,columns=['cluster ' + str(i) for i in range(1,11)])
print(MDS_df.head())

In [ ]:
len(MDS_df)

In [ ]:
finalDf = pd.concat([MDS_df, result_strains[['strain']]], axis = 1).dropna()

In [ ]:
finalDf

In [ ]:
import numpy as np
pairwise_distance_array = np.array(dataa)[np.triu_indices(486, k = 1)]

In [ ]:
pairwise_df = pd.DataFrame(pairwise_distance_array)

In [ ]:
from scipy.spatial.distance import squareform, pdist
euclidean_distance_array = pdist(finalDf[['cluster 1','cluster 2']])

In [ ]:
euclidean_distance_array

In [ ]:
len(euclidean_distance_array)

In [ ]:
euclidean_df = pd.DataFrame(euclidean_distance_array)

In [ ]:
final_df = pairwise_df.merge(euclidean_df,how='inner',left_index=True, right_index=True)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
alt.Chart(final_df).mark_circle(size=60).encode(
    x=alt.X('0_x',title = "pairwise distance"),
    y=alt.X('0_y', title = "Euclidean distance"),
    tooltip=['0_x', '0_y']
).properties(title="pairwise vs. Euclidean scatterplot: MDS")

In [ ]:
dataa = pd.read_csv("DistanceMatrixPairwiseEuclidean")

In [ ]:
import umap
reducer = umap.UMAP()
embedding = reducer.fit_transform(dataa)

In [ ]:
UMAP_df = pd.DataFrame(embedding,columns=['cluster ' + str(i) for i in range(1,3)])
print(UMAP_df.head())

In [ ]:
len(UMAP_df)

In [ ]:
finalDf = pd.concat([UMAP_df, result_strains[['strain']]], axis = 1).dropna()

In [ ]:
finalDf

In [ ]:
import numpy as np
pairwise_distance_array = np.array(dataa)[np.triu_indices(486, k = 1)]

In [ ]:
pairwise_df = pd.DataFrame(pairwise_distance_array)

In [ ]:
from scipy.spatial.distance import squareform, pdist
euclidean_distance_array = pdist(finalDf[['cluster 1','cluster 2']])

In [ ]:
euclidean_distance_array

In [ ]:
len(euclidean_distance_array)

In [ ]:
euclidean_df = pd.DataFrame(euclidean_distance_array)

In [ ]:
final_df = pairwise_df.merge(euclidean_df,how='inner',left_index=True, right_index=True)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
alt.Chart(final_df).mark_circle(size=60).encode(
    x=alt.X('0_x',title = "pairwise distance"),
    y=alt.X('0_y', title = "Euclidean distance"),
    tooltip=['0_x', '0_y']
).properties(title="pairwise vs. Euclidean scatterplot: UMAP")

In [ ]:
from sklearn.decomposition import PCA


#performing PCA on my pandas dataframe 
pca = PCA(n_components=10,svd_solver='full') #can specify n, since with no prior knowledge, I use None
principalComponents = pca.fit_transform(dataa)
principalDf = pd.DataFrame(data = principalComponents, columns = ["principal component " + str(i) for i in range(1,11)])

In [ ]:
principalDf.head()

In [ ]:
finalDf = pd.concat([principalDf, result_strains[['strain']]], axis = 1).dropna()

In [ ]:
finalDf

In [ ]:
import numpy as np
pairwise_distance_array = np.array(dataa)[np.triu_indices(486, k = 1)]

In [ ]:
pairwise_df = pd.DataFrame(pairwise_distance_array)

In [ ]:
from scipy.spatial.distance import squareform, pdist
euclidean_distance_array = pdist(finalDf[['principal component 1','principal component 2']])

In [ ]:
euclidean_distance_array

In [ ]:
len(euclidean_distance_array)

In [ ]:
euclidean_df = pd.DataFrame(euclidean_distance_array)

In [ ]:
final_df = pairwise_df.merge(euclidean_df,how='inner',left_index=True, right_index=True)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
alt.Chart(final_df).mark_circle(size=60).encode(
    x=alt.X('0_x',title = "pairwise distance"),
    y=alt.X('0_y', title = "Euclidean distance"),
    tooltip=['0_x', '0_y']
).properties(title="pairwise vs. Euclidean scatterplot: PCA")

# Creating a method

In [ ]:
"""
similarity_matrix: matrix of pairwise differences from data (pandas Dataframe)
df_merged: the mergd dataframe between your tree JSON and the FASTA file strains (pandas Dataframe)
type_of_embedding: PCA, UMAP, MDS, or TSNE (string)
"""
def scatterplot_tooltips(similarity_matrix, df_merged, type_of_embedding):
    import numpy as np
    from scipy.spatial.distance import squareform, pdist
    import altair as alt
    
    if type_of_embedding == "UMAP":
        import umap
        reducer = umap.UMAP()
        embedding = reducer.fit_transform(dataa)
        embedding_df = pd.DataFrame(embedding,columns=['cluster ' + str(i) for i in range(1,3)])
    elif type_of_embedding == "MDS":
        from sklearn.manifold import MDS
        embedding = MDS(n_components=10,metric=True,dissimilarity='precomputed')
        X_transformed = embedding.fit_transform(dataa)
        embedding_df = pd.DataFrame(X_transformed,columns=['cluster ' + str(i) for i in range(1,11)])
    elif type_of_embedding == "TSNE":
        from sklearn.manifold import TSNE
        embedding = TSNE(n_components=2,metric='precomputed',perplexity = 25.95)
        X_transformed = embedding.fit_transform(dataa)
        embedding_df = pd.DataFrame(X_transformed,columns=['cluster ' + str(i) for i in range(1,3)])
    elif type_of_embedding == "PCA":
        pca = PCA(n_components=10,svd_solver='full') #can specify n, since with no prior knowledge, I use None
        principalComponents = pca.fit_transform(dataa)
        embedding_df = pd.DataFrame(data = principalComponents, columns = ["principal component " + str(i) for i in range(1,11)])
    finalDf = pd.concat([embedding_df, df_merged[['strain']]], axis = 1).dropna()

    pairwise_distance_array = np.array(dataa)[np.triu_indices(486, k = 1)]
    pairwise_df = pd.DataFrame(pairwise_distance_array)
    
    euclidean_distance_array = pdist(finalDf[['cluster 1','cluster 2']])
    euclidean_df = pd.DataFrame(euclidean_distance_array)

    final_df = pairwise_df.merge(euclidean_df,how='inner',left_index=True, right_index=True)
    
    alt.data_transformers.disable_max_rows()
    chart = alt.Chart(final_df).mark_circle(size=60).encode(
        x=alt.X('0_x',title = "pairwise distance"),
        y=alt.X('0_y', title = "Euclidean distance"),
        tooltip=['0_x', '0_y']
    ).properties(title="Pairwise vs. Euclidean scatterplot: " + type_of_embedding)
    return chart   
        

In [ ]:
scatterplot_tooltips(dataa,merged_df,"MDS")

# Creating a method for making the similarity matrix

In [ ]:
def making_scatterplot_pairwise_matrix(path,df_merged)
    import pandas as pd
    import numpy as np
    from Bio import SeqIO
    strains = []
    genomes = []
    for record in SeqIO.parse(path, "fasta"):
            strains.append(str(record.id))
            genomes.append(str(record.seq))
    genomes_df = pd.DataFrame(genomes)
    genomes_df.columns = ['genomes']

    strains_df = pd.DataFrame(strains)
    strains_df.columns = ['strain']

    df_merged = genomes_df.merge(strains_df, how='outer', left_index=True, right_index=True)

    merged2_df = df_merged.merge(merged_df, how = 'inner', on = 'strain')
    merged2_df = merged2_df[['genomes','strain']]
    
    genomes_numpy = merged2_df[['genomes']].to_numpy()
    genomes = genomes_numpy.tolist()
    genomes = list(genomes_numpy)
    
    numbers = []
    matrixOfNum = []
    for i in merged2_df.index:
        for j in merged2_df.index:
            num = hamming_distance(merged2_df['genomes'][i],merged2_df['genomes'][j])
            numbers.append(num)
        matrixOfNum.append(numbers)
        numbers = []
    dataa = pd.DataFrame(matrixOfNum, columns = ["strain " + str(i) for i in range(1,len(matrixOfNum) + 1)], dtype="float")
    return dataa
    
    

In [ ]:
genomes_df = pd.DataFrame(genomes)
genomes_df.columns = ['genomes']

strains_df = pd.DataFrame(strains)
strains_df.columns = ['strain']

df_merged = genomes_df.merge(strains_df, how='outer', left_index=True, right_index=True)

merged2_df = df_merged.merge(merged_df, how = 'inner', on = 'strain')
merged2_df = merged2_df[['genomes','strain']]
merged2_df.head()


In [ ]:
genomes_numpy = merged2_df[['genomes']].to_numpy()
genomes = genomes_numpy.tolist()
genomes = list(genomes_numpy)

In [ ]:
# Return the Hamming distance between string1 and string2.
# string1 and string2 should be the same length.
def hamming_distance(array1, array2): 
    # Start with a distance of zero, and count up
    distance = 0
    # Loop over the indices of the string
    L = len(array1)
    for i in range(L):
        # Add 1 to the distance if these two characters are not equal
        if array1[i] != array2[i] and array1[i] != 'N' and array2[i] != 'N':
            distance += 1
    # Return the final count of differences
    return distance

In [ ]:
#Using the Hamming distance to create a similarity matrix
numbers = []
matrixOfNum = []
for i in merged2_df.index:
    for j in merged2_df.index:
        num = hamming_distance(merged2_df['genomes'][i],merged2_df['genomes'][j])
        numbers.append(num)
    matrixOfNum.append(numbers)
    numbers = []
dataa = pd.DataFrame(matrixOfNum, columns = ["strain " + str(i) for i in range(1,len(matrixOfNum) + 1)], dtype="float")
print(dataa)

In [ ]:
#Read in correct matrix


In [ ]:
dataa = dataa.to_csv("DistanceMatrixPairwiseEuclidean",sep=',',index=False)

In [ ]:
#work on making this work
path = "Data/aligned_cdc_h3n2_ha_2y_cell_hi.fasta"
import pandas as pd
import numpy as np
import altair as alt
from Bio import SeqIO
strains = []
genomes = []
i = 0
dropped_strains = ["A/Usl/3850/2019","A/Navarra/2284/2018", "A/Austria/1123467/2019", "A/Austria/1127203/2019", "A/Catalonia/NSVH100995626/2019", "A/Ontario/RV2296/2019", "A/Sydney/781/2019", 'A/SouthAustralia/1034/2019','A/Usk/3850/2019']
for record in SeqIO.parse(path, "fasta"):
    if(record.id not in dropped_strains):
        strains.append(str(record.id))
        genomes.append(str(record.seq))
        i = i + 1

#### Checking to make sure the file I picked is a aligned Fasta file / is the file I wanted

In [ ]:
len(strains)
print(len(genomes[0]) == len(genomes[1]))
print(len(genomes))

In [ ]:
strains[:5]

# Creating the Distance Matrix
- I used Hamming Distance to find the pairwise distance between each genome and each other genome, effectively creating a similarity/distance matrix
    - In my Hamming Distance method, I only counted something as a different if it was a mismatch between the nucleotides (A,G,C, or T), not gaps (as that was throwing off the algorithm too much for smaller strains)
- I then used Seaborn to generate a heatmap to make sure the matrix looked correct

In [ ]:
#if statement for runing ist matrix
# Return the Hamming distance between string1 and string2.
# string1 and string2 should be the same length.
def hamming_distance(array1, array2): 
    # Start with a distance of zero, and count up
    distance = 0
    # Loop over the indices of the string
    L = len(array1)
    for i in range(L):
        # Add 1 to the distance if these two characters are not equal
        if array1[i] != array2[i] and array1[i] != 'N' and array2[i] != 'N':
            distance += 1
    # Return the final count of differences
    return distance

In [ ]:
#Using the Hamming distance to create a similarity matrix
numbers = []
matrixOfNum = []
for i in range(0, len(genomes)):
    for j in range(0,len(genomes)):
        num = hamming_distance(genomes[i],genomes[j])
        numbers.append(num)
    matrixOfNum.append(numbers)
    numbers = []
similarity_matrix = pd.DataFrame(matrixOfNum, columns = ["strain " + str(i) for i in range(1,len(matrixOfNum) + 1)], dtype="float")

In [ ]:
print(similarity_matrix.head())

In [ ]:
import seaborn as sns
print(sns.heatmap(similarity_matrix))

# Reading in the Metadata
- The metadata is used for getting the region, country, etc of different strains. This data is used to color the clusters.
- The metadata contains all of the possible sampled strains, so many of these will not be in the genomes from the aligned file, probably because the strains were corrupted, too short, etc. 
- We merge this metadata with the strains we have in the aligned file to get a list of all of the strains that match between both. It should come out to the amount of strains in the aligned file.

In [ ]:
#merging my final dataframe with their regions and strain names
pathMeta = 'Data\metadata_h3n2_ha.tsv'
metadata_df = pd.read_csv(pathMeta, delimiter='\t')

In [ ]:
metadata_df.head()

In [ ]:
metadata_df.shape

In [ ]:
#making a dataframe out of the strains from the alignment file to merge with the metadata
strains_df = pd.DataFrame(strains)
strains_df.columns = ['strain']

In [ ]:
strains_df.shape

In [ ]:
#Merging strains and metadata 
result_strains = pd.merge(strains_df,metadata_df, on='strain')

In [ ]:
#checking that no strains were lost
result_strains.shape

In [ ]:
np.setdiff1d(strains_df['strain'].unique(),metadata_df['strain'].unique())

# Creating the Phylogenetic Tree in Altair
- I used Altair to make this tree (Documentation linked [here][1]
- I opened and imported the json from a build from NextStrain ([flu][2], [zika][3], etc)
- The data from the JSON and the Data from the tree are usually a little different, so after merging the two dataframes you may get some errors.

[1]: https://altair-viz.github.io/index.html
[2]: https://github.com/nextstrain/seasonal-flu
[3]: https://altair-viz.github.io/index.html

In [ ]:
import altair as alt
from augur.utils import json_to_tree
import json
import pandas as pd

In [ ]:
# Import modules
with open('Data/flu_seasonal_h3n2_ha_2y_tree.json') as fh:
    json_tree_handle = json.load(fh)
#json_tree_handle = json.loads(requests.get("http://data.nextstrain.org/zika_tree.json").text)
# Download tree JSON file used by auspice. Despite its filename extension, the data are compressed.
#http = urllib3.PoolManager()
# = http.request('GET', "")


In [ ]:
tree = json_to_tree(json_tree_handle)

In [ ]:
tree

In [ ]:
node_data = [
    {
        "strain": node.name,
        "date": node.attr["num_date"],
        "y": node.yvalue,
        "region": node.attr["region"],
        "country": node.attr["country"],
        "parent_date": node.parent is not None and node.parent.attr["num_date"] or node.attr["num_date"],
        "parent_y": node.parent is not None and node.parent.yvalue or node.yvalue,
        "clade_membership" : node.attr['clade_membership']
    }
    for node in tree.find_clades(terminal=True)
]

In [ ]:
node_data[10]

In [ ]:
node_df = pd.DataFrame(node_data)

In [ ]:
node_df.head()

In [ ]:
node_df["y"] = node_df["y"].max() - node_df["y"]

In [ ]:
node_df["parent_y"] = node_df["parent_y"].max() - node_df["parent_y"]

In [ ]:
node_df.shape

In [ ]:
node_df.head()

In [ ]:
node_df["region"].unique()

In [ ]:
node_numpy = node_df['clade_membership'].to_numpy()
clade_ok = []
clades = ['3c3.A', 'A1b/131K', 'A1b/135K', 'A2/re', 'A2', 'A3']
#node_numpy = node_numpy.tolist()
for node in node_numpy:
    if node not in clades:
        clade_ok.append("other")
    else:
        clade_ok.append(node)

In [ ]:
clade_new = pd.DataFrame(clade_ok)
node_df = node_df.merge(clade_new, how='outer', left_index=True, right_index=True)

In [ ]:
node_df.head()

In [ ]:
node_df.columns = ['strain', 'date', 'y', 'region', 'country', 'parent_date',
      'parent_y', 'clade_membership', 'clade_membership_color']

# Running PCA on the Dataset

In [ ]:
from sklearn.decomposition import PCA

#performing PCA on my pandas dataframe 
pca = PCA(n_components=10,svd_solver='full') #can specify n, since with no prior knowledge, I use None
principalComponents = pca.fit_transform(similarity_matrix)
principalDf = pd.DataFrame(data = principalComponents, columns = ["principal component " + str(i) for i in range(1,11)])

In [ ]:
principalDf

# Visualizing PCA in Altair
- Visualizing both PC1 (x) and PC2(y), and PC3(x) and PC4(y) in seperate plots

In [ ]:
"""
principal_Df -- Data from data reduction (-T-SNE, MDS, etc) (pandas DataFrame)
result_metadata -- the metadata that is being read in (Pandas DataFrame)
fields, the parts of metadata that should be concatenated with princiapl_Df (list)
"""
def concatenate_results_with_strain_data(principal_Df,result_metadata,fields):
    finalDf = pd.concat([principal_Df, result_metadata[fields]], axis = 1)
    return finalDf

In [ ]:
finalDf = concatenate_results_with_strain_data(principalDf, node_df, ['strain','clade_membership', 'clade_membership_color'])
strain_df = finalDf['strain']

In [ ]:
import pandas as pd
import altair as alt
import numpy as np
from scipy.spatial.distance import squareform, pdist
import pandas as pd
import numpy as np
from Bio import SeqIO
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from augur.utils import json_to_tree
import json
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
import umap
from scipy.stats import linregress
from pathlib import Path

In [ ]:
#work on making this work
path = "Data/aligned_cdc_h3n2_ha_2y_cell_hi.fasta"
strains = []
genomes = []
dropped_strains = ["A/Usl/3850/2019","A/Navarra/2284/2018", "A/Austria/1123467/2019", "A/Austria/1127203/2019", "A/Catalonia/NSVH100995626/2019", "A/Ontario/RV2296/2019", "A/Sydney/781/2019", 'A/SouthAustralia/1034/2019','A/Usk/3850/2019']
for record in SeqIO.parse(path, "fasta"):
    if(record.id not in dropped_strains):
        strains.append(str(record.id))
        genomes.append(str(record.seq))

In [ ]:
"""
similarity_matrix: matrix of pairwise differences from data (pandas Dataframe)
df_merged: the mergd dataframe between your tree JSON and the FASTA file strains (pandas Dataframe)
column1: one of the data cluster column names in df_merged
column2: the other data cluster column name in df_merged
type_of_embedding: type of embedding (PCA, UMAP, TSNE, MDS)
"""

def scatterplot_tooltips(similarity_matrix, df_merged, column1, column2, type_of_embedding):
    
    embedding_df = df_merged[[column1, column2]]

    finalDf = pd.concat([embedding_df, df_merged[['strain']]], axis = 1).dropna()
    finalDf.columns = [column1, column2, 'strain']
    
    pairwise_distance_array = np.array(similarity_matrix)[np.triu_indices(len(finalDf), k = 0)]
    pairwise_df = pd.DataFrame(pairwise_distance_array)
    pairwise_df = pd.concat([pairwise_df, finalDf[['strain']]], axis = 1)

    euclidean_distance_array = pdist(finalDf[[column1, column2]])
    euclidean_distance_array = squareform(euclidean_distance_array)
    euclidean_distance_array = euclidean_distance_array[np.triu_indices(len(finalDf), k = 0)]
        
    euclidean_df = pd.DataFrame(euclidean_distance_array)
    euclidean_df = pd.concat([euclidean_df, finalDf[['strain']]], axis = 1).dropna()
    
    final_df = euclidean_df.merge(pairwise_df,how='inner',on = 'strain')

    print(linregress(pairwise_distance_array,euclidean_distance_array))
    
    alt.data_transformers.disable_max_rows()
    chart = alt.Chart(final_df).mark_circle(size=60).encode(
        x=alt.X('0_x',title = "pairwise distance"),
        y=alt.X('0_y', title = "Euclidean distance"),
        tooltip=['strain']
    ).properties(title="Pairwise vs. Euclidean scatterplot: " + type_of_embedding,height=200,width=300)
    return chart   
        

In [ ]:
scatterplot_tooltips(similarity_matrix, finalDf.dropna(), "principal component 1", "principal component 2", "PCA")

In [ ]:
"""Create a box/violin plot in order to plot the within vs between clade distance for 
1) genetic distance and 
2) the different embeddings (PCA, t-SNE, MDS, UMAP) for Zika and Flu"""

In [ ]:
#Plot between vs within clade distance for Pairwise distance

"""Steps:
1) use the similarity matrix and categorize by clade (if within clade, color distance with red, if between clade, use blue)
2) find the euclidean distance matrix and categorize by clade (if within clade, color distance with red, if between cladem use blue)


"""

In [ ]:
similarity_matrix.head()

In [ ]:
similarity_matrix_df = pd.DataFrame(similarity_matrix)
similarity_matrix_df

In [ ]:
strain_df = pd.DataFrame(strains)
strains_similarity_df_merged = similarity_matrix_df.merge(strain_df, how='outer', left_index=True, right_index=True)

In [ ]:
strains_similarity_df_merged

In [ ]:
#strains_df.set_index(similarity_matrix.mean().index)
strain_df.columns = ['strain']

In [ ]:
node_df_new = node_df.merge(strain_df,how = "inner",on="strain")
node_df_new

In [ ]:
node_df_new.set_index(similarity_matrix.mean().index)

In [ ]:
matrix_of_numbers = []
row_of_numbers = []
node_list = node_df_new['clade_membership'].tolist()

for index in node_list:
    for index2 in node_list:
        if(index == index2):
            row_of_numbers.append("within_clade")
        else:
            row_of_numbers.append("between_clade")
    matrix_of_numbers.append(row_of_numbers)
    row_of_numbers = []

len(matrix_of_numbers[0])

In [ ]:
pairwise_distance_array = np.array(similarity_matrix)[np.triu_indices(len(matrix_of_numbers), k = 0)]
pairwise_df = pd.DataFrame(pairwise_distance_array)

color_distance_array = np.array(matrix_of_numbers)[np.triu_indices(len(matrix_of_numbers), k = 0)]
color_df = pd.DataFrame(color_distance_array)

color_similarity = pairwise_df.merge(color_df, right_index = True, left_index = True)

In [ ]:
color_similarity.columns = ['distance','color']

In [ ]:
alt.Chart(color_similarity.sample(n=100000),height=150).mark_boxplot().encode(
    x = alt.X('distance',title="mean of pairwise distances"),
    y = 'color'
)

In [ ]:
#make a method for euclidean distance boxplot

In [ ]:
principalDf

In [ ]:
finalDf

In [ ]:
euclidean_distance_array = pdist(principalDf[['principal component 1', 'principal component 2']])
euclidean_distance_array = squareform(euclidean_distance_array)
euclidean_distance_array = euclidean_distance_array[np.triu_indices(len(matrix_of_numbers), k = 0)]

In [ ]:
euclidean_df = pd.DataFrame(euclidean_distance_array)
euclidean_df

In [ ]:
color_similarity = euclidean_df.merge(color_df, right_index = True, left_index = True)
color_similarity.columns = ['distance','color']

In [ ]:
alt.Chart(color_similarity.sample(n=250000),height=150).mark_boxplot().encode(
    x = alt.X('distance',title="mean of pairwise distances"),
    y = 'color'
)

In [ ]:
#making method for boxplots

In [ ]:
#merged_df MUST include clade_membership and strain name merged with the respective distances
def making_with_vs_between_boxplots(merged_df,column1,column2):
    matrix_of_numbers = []
    row_of_numbers = []
    node_list = merged_df['clade_membership'].tolist()

    for index in node_list:
        for index2 in node_list:
            if(index == index2):
                row_of_numbers.append("within_clade")
            else:
                row_of_numbers.append("between_clade")
        matrix_of_numbers.append(row_of_numbers)
        row_of_numbers = []

    color_distance_array = np.array(matrix_of_numbers)[np.triu_indices(len(matrix_of_numbers), k = 0)]
    color_df = pd.DataFrame(color_distance_array)
    
    euclidean_distance_array = pdist(merged_df[[column1, column2]])
    euclidean_distance_array = squareform(euclidean_distance_array)
    euclidean_distance_array = euclidean_distance_array[np.triu_indices(len(matrix_of_numbers), k = 0)]
    euclidean_df = pd.DataFrame(euclidean_distance_array)
    
    color_similarity = euclidean_df.merge(color_df, right_index = True, left_index = True)
    color_similarity.columns = ['distance','color']
    
    chart = alt.Chart(color_similarity.sample(n=250000),height=150).mark_boxplot().encode(
    x = alt.X('distance',title="mean distances"),
    y = 'color'
    )
    return chart

In [ ]:
making_with_vs_between_boxplots(node_df.merge(finalDf, on='strain'),'TSNE cluster 1','TSNE cluster 2')